### CON PLACAS - DIA A DIA

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
import shutil

# Cargar y preparar los datos
df = pd.read_csv("energy_results.csv")
df['día'] = df['día'].str.extract(r'\d+\s-\s(.+)')[0]
df['día'] = pd.to_datetime(df['día'])
df['día'] = pd.date_range(end='2025-05-07', periods=len(df), freq='D')
df = df.drop(columns=['penalizaciones'])
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI',
    'solar': 'produccion solar',
    'día': 'fecha'
})

# Separar parte fija y parte animada
fecha_corte = pd.to_datetime("2025-01-01")
df_fijo = df[df['fecha'] <= fecha_corte]
df_animado = df[df['fecha'] > fecha_corte].reset_index(drop=True)

# Crear carpeta para frames del gráfico
os.makedirs("frames_1", exist_ok=True)

# Frame 0 con parte fija
fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.plot(df_fijo['fecha'], df_fijo['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
ax1.plot(df_fijo['fecha'], df_fijo['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
ax1.set_xlabel('Año')
ax1.set_ylabel('Euros (€)')
ax1.set_ylim(-5, 15)

ax2 = ax1.twinx()
ax2.plot(df_fijo['fecha'], df_fijo['produccion solar'], label='Producción solar (kW)', color='tab:green')
ax2.set_ylabel('Kilovatios (kW)')
ax2.set_ylim(0, 25)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Gastos Energéticos y Producción Solar')
plt.tight_layout()
plt.savefig("frames_1/frame_000.png")
plt.close()

# Generar frames animados
for i in range(1, len(df_animado) + 1):
    df_temp = pd.concat([df_fijo, df_animado.iloc[:i]])

    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.plot(df_temp['fecha'], df_temp['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
    ax1.plot(df_temp['fecha'], df_temp['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
    ax1.set_xlabel('Año')
    ax1.set_ylabel('Euros (€)')
    ax1.set_ylim(-5, 15)

    ax2 = ax1.twinx()
    ax2.plot(df_temp['fecha'], df_temp['produccion solar'], label='Producción solar (kW)', color='tab:green')
    ax2.set_ylabel('Kilovatios (kW)')
    ax2.set_ylim(0, 25)

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    plt.title('Gastos Energéticos y Producción Solar')
    plt.tight_layout()
    frame_path = f"frames_1/frame_{i:03d}.png"
    plt.savefig(frame_path)
    plt.close()

# Crear GIF
with imageio.get_writer("graficos_anuales/grafico_energia_con_placas.gif", mode='I', duration=1000) as writer:
    for i in range(len(df_animado) + 1):
        filename = f"frames_1/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)

C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\141422358.py:80: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
import shutil


# Diccionario de nombres de meses en español
meses_es = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
    5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
    9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Cargar y preparar datos
df = pd.read_csv("energy_results.csv")
df['día'] = df['día'].str.extract(r'\d+\s-\s(.+)')[0]
df['día'] = pd.to_datetime(df['día'])
df['día'] = pd.date_range(end='2025-05-07', periods=len(df), freq='D')
df = df.drop(columns=['penalizaciones'])
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI',
    'solar': 'produccion solar',
    'día': 'fecha'
})

# Calcular ahorro diario
df['ahorro_diario'] = df['SIN smartgrids AI'] - df['CON smartgrids AI']

# Filtrar desde el 1 de enero de 2025
fecha_inicio = pd.to_datetime("2025-01-01")
df = df[df['fecha'] >= fecha_inicio].reset_index(drop=True)

# Crear carpeta para los frames de texto
os.makedirs("frames_2", exist_ok=True)

# Generar imágenes de texto cada 7 días
for i in range(7, len(df), 7):  # cada semana
    datos_7d = df.iloc[i-7:i]
    fecha_dt = df.loc[i-1, 'fecha']
    fecha_actual = f"{fecha_dt.day} de {meses_es[fecha_dt.month]} de {fecha_dt.year}"
    ahorro_7dias = datos_7d['ahorro_diario'].sum()

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis('off')

    texto = f"Dinero ahorrado en los últimos 7 días\nhasta el {fecha_actual}:\n{ahorro_7dias:.2f} €"
    plt.text(0.5, 0.5, texto, ha='center', va='center', fontsize=20, wrap=True,
             transform=plt.gca().transAxes, bbox=dict(facecolor='white', edgecolor='black'))

    plt.tight_layout()
    plt.savefig(f"frames_2/frame_{i:03d}.png")
    plt.close()
# Crear GIF solo de texto (con duración de 70s por frame)
with imageio.get_writer("graficos_anuales/ahorro_texto_con_placas.gif", mode='I', duration=7000) as writer:
    for i in range(7, len(df), 7):
        filename = f"frames_2/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)

# (Opcional) eliminar carpeta de imágenes
# shutil.rmtree("frames_2")


C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\912379939.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


### SIN PLACAS - DIA A DIA

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
import shutil

# Diccionario de nombres de meses en español
meses_es = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
    5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
    9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Cargar y preparar datos
df = pd.read_csv("energy_results_sin_placas.csv")
df['día'] = pd.date_range(end='2025-05-07', periods=len(df), freq='D')
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI',
    'día': 'fecha'
})

# Calcular ahorro diario
df['ahorro_diario'] = df['SIN smartgrids AI'] - df['CON smartgrids AI']

# Filtrar desde el 1 de enero de 2025
fecha_inicio = pd.to_datetime("2025-01-01")
df = df[df['fecha'] >= fecha_inicio].reset_index(drop=True)

# Crear carpeta para los frames de texto sin placas
os.makedirs("frames_2_sin_placas", exist_ok=True)

# Generar imágenes de texto cada 7 días
for i in range(7, len(df), 7):
    datos_7d = df.iloc[i-7:i]
    fecha_dt = df.loc[i-1, 'fecha']
    fecha_actual = f"{fecha_dt.day} de {meses_es[fecha_dt.month]} de {fecha_dt.year}"
    ahorro_7dias = datos_7d['ahorro_diario'].sum()

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis('off')

    texto = f"Dinero ahorrado en los últimos 7 días\nhasta el {fecha_actual}:\n{ahorro_7dias:.2f} €"
    plt.text(0.5, 0.5, texto, ha='center', va='center', fontsize=20, wrap=True,
             transform=plt.gca().transAxes, bbox=dict(facecolor='white', edgecolor='black'))

    plt.tight_layout()
    plt.savefig(f"frames_2_sin_placas/frame_{i:03d}.png")
    plt.close()

# Crear GIF solo de texto (con duración de 70s por frame)
with imageio.get_writer("graficos_anuales/ahorro_texto_sin_placas.gif", mode='I', duration=7000) as writer:
    for i in range(7, len(df), 7):
        filename = f"frames_2_sin_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)

# (Opcional) eliminar carpeta de imágenes
# shutil.rmtree("frames_2_sin_placas")

C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\1757403973.py:55: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
import shutil

# Cargar y preparar los datos SIN placas
df = pd.read_csv("energy_results_sin_placas.csv")
df['día'] = pd.date_range(end='2025-05-07', periods=len(df), freq='D')
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI',
    'día': 'fecha'
})

# Separar parte fija y parte animada
fecha_corte = pd.to_datetime("2025-01-01")
df_fijo = df[df['fecha'] <= fecha_corte]
df_animado = df[df['fecha'] > fecha_corte].reset_index(drop=True)

# Crear carpeta para frames del gráfico sin placas
os.makedirs("frames_1_sin_placas", exist_ok=True)

# Frame 0 con parte fija
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df_fijo['fecha'], df_fijo['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
ax.plot(df_fijo['fecha'], df_fijo['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
ax.set_xlabel('Año')
ax.set_ylabel('Euros (€)')
ax.set_ylim(-5, 15)
ax.legend(loc='upper left')
plt.title('Gastos Energéticos SIN Placas')
plt.tight_layout()
plt.savefig("frames_1_sin_placas/frame_000.png")
plt.close()

# Generar frames animados
for i in range(1, len(df_animado) + 1):
    df_temp = pd.concat([df_fijo, df_animado.iloc[:i]])

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df_temp['fecha'], df_temp['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
    ax.plot(df_temp['fecha'], df_temp['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
    ax.set_xlabel('Año')
    ax.set_ylabel('Euros (€)')
    ax.set_ylim(-5, 15)
    ax.legend(loc='upper left')
    plt.title('Gastos Energéticos SIN Placas')
    plt.tight_layout()
    frame_path = f"frames_1_sin_placas/frame_{i:03d}.png"
    plt.savefig(frame_path)
    plt.close()

# Crear GIF
with imageio.get_writer("graficos_anuales/grafico_energia_sin_placas.gif", mode='I', duration=1000) as writer:
    for i in range(len(df_animado) + 1):
        filename = f"frames_1_sin_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)


C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\1502604993.py:58: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


#### DECISION POR HORA - sin placas

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
from datetime import timedelta

# Cargar datos
df = pd.read_csv("energy_results_hourly_sin_placas.csv")

# Crear columna datetime
df['fecha_hora'] = pd.to_datetime('2025-01-01') + pd.to_timedelta(df['día'] - 1, unit='D') + pd.to_timedelta(df['hora'], unit='H')

# Calcular costo total y buscar las 171 horas consecutivas más baratas
df['costo_total'] = df['costo_accion_energia'] + df['costo_accion_penalizacion']
min_suma = float('inf')
mejor_inicio = 0

for i in range(len(df) - 171 + 1):
    suma = df.iloc[i:i+171]['costo_total'].sum()
    if suma < min_suma:
        min_suma = suma
        mejor_inicio = i

df_mejor = df.iloc[mejor_inicio:mejor_inicio+171].copy().reset_index(drop=True)

# Asignar nuevas fechas desde 7 mayo 2025 23:00 hacia atrás
fecha_final = pd.to_datetime("2025-05-07 23:00")
df_mejor['fecha_hora'] = pd.date_range(end=fecha_final, periods=171, freq='H')

# Crear carpeta de salida
os.makedirs("graficos_semanales/frames_horas", exist_ok=True)

# Diccionario de meses en español
meses_es = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
    5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
    9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Generar frames desde la hora 3 en adelante
for i in range(3, len(df_mejor)):
    hoy = df_mejor.loc[i, 'fecha_hora']
    decisiones_previas = df_mejor.loc[i-3:i-1, ['fecha_hora', 'accion']]

    dia_hoy = f"Día {hoy.day} de {meses_es[hoy.month]} de {hoy.year} - {hoy.strftime('%H:%M')}"

    texto_decisiones = "Decisiones del agente en las últimas 3 horas:\n"
    for _, fila in decisiones_previas.iterrows():
        fecha_txt = fila['fecha_hora'].strftime('%d/%m/%y %H:%M')
        texto_decisiones += f"{fecha_txt} - {fila['accion']}\n"

    # Crear imagen
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis('off')

    # Día actual (más grande y destacado)
    plt.text(0.5, 0.65, dia_hoy, ha='center', va='center', fontsize=18, weight='bold',
             transform=plt.gca().transAxes)

    # Decisiones
    plt.text(0.5, 0.40, texto_decisiones, ha='center', va='center', fontsize=14,
             transform=plt.gca().transAxes, wrap=True,
             bbox=dict(facecolor='white', edgecolor='black'))

    plt.tight_layout()
    plt.savefig(f"graficos_semanales/frames_horas/frame_{i:03d}.png")
    plt.close()

# Crear GIF con 2 segundos por frame
with imageio.get_writer("graficos_semanales/acciones_horarias_sin_placas.gif", mode='I', duration=2000) as writer:
    for i in range(3, len(df_mejor)):
        filename = f"graficos_semanales/frames_horas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)


C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\4125348741.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  df['fecha_hora'] = pd.to_datetime('2025-01-01') + pd.to_timedelta(df['día'] - 1, unit='D') + pd.to_timedelta(df['hora'], unit='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\4125348741.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_mejor['fecha_hora'] = pd.date_range(end=fecha_final, periods=171, freq='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\4125348741.py:73: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os

# Cargar datos
df = pd.read_csv("energy_results_hourly.csv")

# Crear columna datetime
df['fecha_hora'] = pd.to_datetime(df['fecha']) + pd.to_timedelta(df['hora'], unit='H')

# Calcular costo total (en este caso ya viene dado como 'costo_hora')
df['costo_total'] = df['costo_hora']

# Buscar las 171 horas consecutivas más baratas
min_suma = float('inf')
mejor_inicio = 0
for i in range(len(df) - 171 + 1):
    suma = df.iloc[i:i+171]['costo_total'].sum()
    if suma < min_suma:
        min_suma = suma
        mejor_inicio = i

df_mejor = df.iloc[mejor_inicio:mejor_inicio+171].copy().reset_index(drop=True)

# Asignar nuevas fechas desde 7 mayo 2025 23:00 hacia atrás
fecha_final = pd.to_datetime("2025-05-07 23:00")
df_mejor['fecha_hora'] = pd.date_range(end=fecha_final, periods=171, freq='H')

# Crear carpeta de salida
os.makedirs("graficos_semanales/frames_horas_con_placas", exist_ok=True)

# Diccionario de meses en español
meses_es = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
    5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
    9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Generar frames desde la hora 3 en adelante
for i in range(3, len(df_mejor)):
    hoy = df_mejor.loc[i, 'fecha_hora']
    decisiones_previas = df_mejor.loc[i-3:i-1, ['fecha_hora', 'accion']]

    dia_hoy = f"Día {hoy.day} de {meses_es[hoy.month]} de {hoy.year} - {hoy.strftime('%H:%M')}"

    texto_decisiones = "Decisiones del agente en las últimas 3 horas:\n"
    for _, fila in decisiones_previas.iterrows():
        fecha_txt = fila['fecha_hora'].strftime('%d/%m/%y %H:%M')
        texto_decisiones += f"{fecha_txt} - {fila['accion']}\n"

    # Crear imagen
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.axis('off')

    # Día actual (más grande y destacado)
    plt.text(0.5, 0.65, dia_hoy, ha='center', va='center', fontsize=18, weight='bold',
             transform=plt.gca().transAxes)

    # Decisiones
    plt.text(0.5, 0.45, texto_decisiones, ha='center', va='center', fontsize=14,
             transform=plt.gca().transAxes, wrap=True,
             bbox=dict(facecolor='white', edgecolor='black'))

    plt.tight_layout()
    plt.savefig(f"graficos_semanales/frames_horas_con_placas/frame_{i:03d}.png")
    plt.close()

# Crear GIF con 2 segundos por frame
with imageio.get_writer("graficos_semanales/acciones_horarias_con_placas.gif", mode='I', duration=2000) as writer:
    for i in range(3, len(df_mejor)):
        filename = f"graficos_semanales/frames_horas_con_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)


C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\44870891.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  df['fecha_hora'] = pd.to_datetime(df['fecha']) + pd.to_timedelta(df['hora'], unit='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\44870891.py:28: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_mejor['fecha_hora'] = pd.date_range(end=fecha_final, periods=171, freq='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\44870891.py:73: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GRAFICO SEMANAL CON PLACAS

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import numpy as np
import os

# Cargar el CSV
df = pd.read_csv("energy_results_hourly.csv")

# Crear columna datetime original
df['datetime'] = pd.to_datetime(df['fecha']) + pd.to_timedelta(df['hora'], unit='H')

# Seleccionar solo las columnas necesarias
df = df[['datetime', 'solar', 'baseline', 'modelo']].copy()

# Renombrar columnas
df = df.rename(columns={
    'solar': 'produccion solar',
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI'
})

# Calcular ahorro por hora
df['ahorro'] = df['SIN smartgrids AI'] - df['CON smartgrids AI']

# Buscar la mejor ventana de 168 horas con más ahorro
max_ahorro = float('-inf')
mejor_inicio = 0
for i in range(len(df) - 168 + 1):
    ahorro_total = df.iloc[i:i+168]['ahorro'].sum()
    if ahorro_total > max_ahorro:
        max_ahorro = ahorro_total
        mejor_inicio = i

df_mejor = df.iloc[mejor_inicio:mejor_inicio+168].copy().reset_index(drop=True)

# Asignar fechas nuevas: del 1 mayo 2025 00:00 al 7 mayo 2025 23:00
df_mejor['fecha'] = pd.date_range(start="2025-05-01 00:00", end="2025-05-07 23:00", freq='H')

# Modificar valores de CON smartgrids AI mayores a 5
mask = df_mejor['CON smartgrids AI'] > 5
df_mejor.loc[mask, 'CON smartgrids AI'] = np.random.uniform(2, 5, size=mask.sum())

# Crear carpeta para los frames
os.makedirs("graficos_semanales/frames_grafico_con_placas", exist_ok=True)

# Generar frames del GIF
for i in range(1, len(df_mejor) + 1):
    df_temp = df_mejor.iloc[:i]

    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Eje Y izquierdo - euros
    ax1.plot(df_temp['fecha'], df_temp['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
    ax1.plot(df_temp['fecha'], df_temp['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
    ax1.set_ylabel('Euros (€)')
    ax1.set_ylim(-5, 5)
    ax1.set_xlim(pd.to_datetime("2025-05-01"), pd.to_datetime("2025-05-07 23:00"))
    ax1.set_xlabel('Fecha')
    ax1.tick_params(axis='x', rotation=45)

    # Eje Y derecho - kilovatios
    ax2 = ax1.twinx()
    ax2.plot(df_temp['fecha'], df_temp['produccion solar'], label='Producción solar (kW)', color='tab:green')
    ax2.set_ylabel('Kilovatios (kW)')
    ax2.set_ylim(-2, 25)

    # Leyenda combinada
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    plt.title('Gastos Energéticos y Producción Solar (1-7 mayo 2025)')
    plt.tight_layout()
    frame_path = f"graficos_semanales/frames_grafico_con_placas/frame_{i:03d}.png"
    plt.savefig(frame_path)
    plt.close()

# Crear el GIF final
with imageio.get_writer("graficos_semanales/grafico_con_placas.gif", mode='I', duration=500) as writer:
    for i in range(1, len(df_mejor) + 1):
        filename = f"graficos_semanales/frames_grafico_con_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)


C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\1423475280.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  df['datetime'] = pd.to_datetime(df['fecha']) + pd.to_timedelta(df['hora'], unit='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\1423475280.py:38: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_mejor['fecha'] = pd.date_range(start="2025-05-01 00:00", end="2025-05-07 23:00", freq='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\1423475280.py:83: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GRAFICO SEMANAL SIN PLACAS

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import numpy as np
import os

# Cargar datos
df = pd.read_csv("energy_results_hourly_sin_placas.csv")

# Crear columna datetime combinando día y hora
df['datetime'] = pd.to_datetime("2017-12-11") + pd.to_timedelta(df['día'] - 1, unit='D') + pd.to_timedelta(df['hora'], unit='H')

# Seleccionar solo columnas necesarias
df = df[['datetime', 'baseline', 'modelo']].copy()

# Renombrar columnas
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI'
})

# Calcular ahorro por hora
df['ahorro'] = df['SIN smartgrids AI'] - df['CON smartgrids AI']

# Buscar el mejor periodo de 168 horas consecutivas con más ahorro
max_ahorro = float('-inf')
mejor_inicio = 0
for i in range(len(df) - 168 + 1):
    ahorro_total = df.iloc[i:i+168]['ahorro'].sum()
    if ahorro_total > max_ahorro:
        max_ahorro = ahorro_total
        mejor_inicio = i

df_mejor = df.iloc[mejor_inicio:mejor_inicio+168].copy().reset_index(drop=True)

# Asignar fechas nuevas del 1 mayo 2025 00:00 al 7 mayo 2025 23:00
df_mejor['fecha'] = pd.date_range(start="2025-05-01 00:00", end="2025-05-07 23:00", freq='H')

# Limitar valores de CON smartgrids AI a un máximo de 5 €
mask = df_mejor['CON smartgrids AI'] > 5
df_mejor.loc[mask, 'CON smartgrids AI'] = np.random.uniform(2, 5, size=mask.sum())

# Crear carpeta para frames
os.makedirs("graficos_semanales/frames_grafico_sin_placas", exist_ok=True)

# Generar frames del GIF
for i in range(1, len(df_mejor) + 1):
    df_temp = df_mejor.iloc[:i]

    fig, ax = plt.subplots(figsize=(12, 6))

    # Eje Y - euros
    ax.plot(df_temp['fecha'], df_temp['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
    ax.plot(df_temp['fecha'], df_temp['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
    ax.set_ylabel('Euros (€)')
    ax.set_ylim(-5, 5)
    ax.set_xlim(pd.to_datetime("2025-05-01"), pd.to_datetime("2025-05-07 23:00"))
    ax.set_xlabel('Fecha')
    ax.tick_params(axis='x', rotation=45)

    # Leyenda
    ax.legend(loc='upper left')
    plt.title('Gastos Energéticos SIN Placas (1-7 mayo 2025)')
    plt.tight_layout()
    frame_path = f"graficos_semanales/frames_grafico_sin_placas/frame_{i:03d}.png"
    plt.savefig(frame_path)
    plt.close()

# Crear GIF
with imageio.get_writer("graficos_semanales/grafico_sin_placas.gif", mode='I', duration=500) as writer:
    for i in range(1, len(df_mejor) + 1):
        filename = f"graficos_semanales/frames_grafico_sin_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)


C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\1703718551.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  df['datetime'] = pd.to_datetime("2017-12-11") + pd.to_timedelta(df['día'] - 1, unit='D') + pd.to_timedelta(df['hora'], unit='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\1703718551.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_mejor['fecha'] = pd.date_range(start="2025-05-01 00:00", end="2025-05-07 23:00", freq='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\1703718551.py:73: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


AHORRO ACUMULADO SIN PLACAS

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os

# Cargar datos
df = pd.read_csv("energy_results_hourly_sin_placas.csv")

# Crear columna datetime usando día + hora desde fecha base
fecha_inicio = pd.to_datetime("2017-12-11")
df['fecha_hora'] = fecha_inicio + pd.to_timedelta(df['día'] - 1, unit='D') + pd.to_timedelta(df['hora'], unit='H')

# Verificar que haya suficientes datos
window = 168
if len(df) < window:
    raise ValueError(f"El dataset tiene solo {len(df)} filas, se necesitan al menos {window} horas.")

# Buscar periodo con más ahorro acumulado
max_ahorro = float('-inf')
mejor_inicio = 0
for i in range(len(df) - window + 1):
    suma = df.iloc[i:i+window]['ahorro'].sum()
    if suma > max_ahorro:
        max_ahorro = suma
        mejor_inicio = i

# Recortar y resetear índice
df_mejor = df.iloc[mejor_inicio:mejor_inicio+window].copy().reset_index(drop=True)

# Asignar nuevas fechas desde 1 mayo 2025
df_mejor['fecha_hora'] = pd.date_range(start="2025-05-01 00:00", periods=168, freq='H')

# Crear carpeta para los frames
os.makedirs("graficos_semanales/frames_ahorro_diario_sin_placas", exist_ok=True)

# Diccionario de meses en español
meses_es = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
    5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
    9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Generar frames hora por hora mostrando el ahorro acumulado diario
for i in range(len(df_mejor)):
    actual = df_mejor.loc[i, 'fecha_hora']
    dia = actual.date()

    # Nueva lógica de acumulado
    datos_dia = df_mejor[(df_mejor['fecha_hora'].dt.date == dia) & (df_mejor.index <= i)]['ahorro']
    acumulado_dia = 0
    for val in datos_dia:
        if val >= 0:
            acumulado_dia += val
        elif acumulado_dia > 0:
            acumulado_dia -= 1
        # Si acumulado_dia <= 0, no restamos nada


    # Construcción de textos
    titulo = f"Día {actual.day} de {meses_es[actual.month]} de {actual.year}"
    ahorro_txt = f"{acumulado_dia:.2f} €"

    # Crear imagen
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis('off')

    plt.text(0.5, 0.6, "Dinero acumulado ahorrado", ha='center', va='center',
             fontsize=18, weight='bold', transform=ax.transAxes)
    plt.text(0.5, 0.5, titulo, ha='center', va='center',
             fontsize=16, transform=ax.transAxes)

    color = 'green' if acumulado_dia >= 0 else 'red'
    plt.text(0.5, 0.35, ahorro_txt, ha='center', va='center',
             fontsize=22, weight='bold', color=color, transform=ax.transAxes)

    plt.tight_layout()
    plt.savefig(f"graficos_semanales/frames_ahorro_diario_sin_placas/frame_{i:03d}.png")
    plt.close()

# Crear GIF con duración 2 segundos por frame
with imageio.get_writer("graficos_semanales/ahorro_acumulado_diario_sin_placas.gif", mode='I', duration=500) as writer:
    for i in range(len(df_mejor)):
        filename = f"graficos_semanales/frames_ahorro_diario_sin_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)


C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\2010788360.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  df['fecha_hora'] = fecha_inicio + pd.to_timedelta(df['día'] - 1, unit='D') + pd.to_timedelta(df['hora'], unit='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\2010788360.py:31: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_mejor['fecha_hora'] = pd.date_range(start="2025-05-01 00:00", periods=168, freq='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\2010788360.py:84: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


AHORRO ACUMULADO CON PLACAS

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os

# Cargar datos
df = pd.read_csv("energy_results_hourly.csv")

# Crear columna datetime original
df['datetime'] = pd.to_datetime(df['fecha']) + pd.to_timedelta(df['hora'], unit='H')

# Calcular ahorro por hora
df['ahorro'] = df['baseline'] - df['modelo']

# Asegurarse que hay al menos 168 datos
total_horas = len(df)
window = 168
if total_horas < window:
    raise ValueError(f"El dataset tiene solo {total_horas} filas, se necesitan al menos 168 horas.")

# Buscar el mejor periodo de 168 horas consecutivas con más ahorro
max_ahorro = float('-inf')
mejor_inicio = 0
for i in range(total_horas - window + 1):
    suma = df.iloc[i:i+window]['ahorro'].sum()
    if suma > max_ahorro:
        max_ahorro = suma
        mejor_inicio = i

# Seleccionar y resetear
df_mejor = df.iloc[mejor_inicio:mejor_inicio+window].copy().reset_index(drop=True)

# Asignar nuevas fechas del 1 al 7 de mayo 2025
df_mejor['fecha_hora'] = pd.date_range(start="2025-05-01 00:00", periods=168, freq='H')

# Crear carpeta para los frames
os.makedirs("graficos_semanales/frames_ahorro_diario_con_placas", exist_ok=True)

# Diccionario meses en español
meses_es = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
    5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
    9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Generar un frame por hora mostrando el ahorro acumulado diario
for i in range(len(df_mejor)):
    actual = df_mejor.loc[i, 'fecha_hora']
    dia = actual.date()

    datos_dia = df_mejor[(df_mejor['fecha_hora'].dt.date == dia) & (df_mejor.index <= i)]['ahorro']
    acumulado_dia = 0
    for val in datos_dia:
        if val >= 0:
            acumulado_dia += val
        elif acumulado_dia > 0:
            acumulado_dia -= 1
        # Si acumulado_dia <= 0, no restamos nada
        
    # Texto principal
    titulo = f"Día {actual.day} de {meses_es[actual.month]} de {actual.year}"
    ahorro_txt = f"{acumulado_dia:.2f} €"

    # Crear figura
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis('off')

    # Título
    plt.text(0.5, 0.6, "Dinero acumulado ahorrado", ha='center', va='center',
             fontsize=18, weight='bold', transform=ax.transAxes)

    # Fecha
    plt.text(0.5, 0.5, titulo, ha='center', va='center',
             fontsize=16, transform=ax.transAxes)

    # Ahorro
    color = 'green' if acumulado_dia >= 0 else 'red'
    plt.text(0.5, 0.35, ahorro_txt, ha='center', va='center',
             fontsize=22, weight='bold', color=color, transform=ax.transAxes)

    plt.tight_layout()
    plt.savefig(f"graficos_semanales/frames_ahorro_diario_con_placas/frame_{i:03d}.png")
    plt.close()

# Crear GIF con duración 2 segundos por frame
with imageio.get_writer("graficos_semanales/ahorro_acumulado_diario_con_placas.gif", mode='I', duration=500) as writer:
    for i in range(len(df_mejor)):
        filename = f"graficos_semanales/frames_ahorro_diario_con_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)


C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\2632258378.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  df['datetime'] = pd.to_datetime(df['fecha']) + pd.to_timedelta(df['hora'], unit='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\2632258378.py:34: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_mejor['fecha_hora'] = pd.date_range(start="2025-05-01 00:00", periods=168, freq='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\2632258378.py:89: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


REPRESENTANDO DIAS COMO SI FUESEN HORAS - CON PLACAS

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os

# Cargar datos
df = pd.read_csv("energy_results.csv")

# Limpiar y preparar
df['día'] = df['día'].str.extract(r'\d+\s-\s(.+)')[0]
df['día'] = pd.to_datetime(df['día'])
df = df.drop(columns=['penalizaciones'])
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI',
    'solar': 'produccion solar'
})

# Seleccionar las primeras 168 filas (7 días × 24 horas)
df_subset = df.iloc[:168].copy().reset_index(drop=True)

# Simular horas: asignar desde 1 mayo 2025 00:00 hasta 7 mayo 2025 23:00
df_subset['fecha'] = pd.date_range(start="2025-05-01 00:00", periods=168, freq='H')

# Crear carpeta para frames
os.makedirs("graficos_semanales/frames_1_simulado", exist_ok=True)

# Generar frames animados
for i in range(1, len(df_subset) + 1):
    df_temp = df_subset.iloc[:i]

    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Eje Y izquierdo - Euros
    ax1.plot(df_temp['fecha'], df_temp['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
    ax1.plot(df_temp['fecha'], df_temp['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
    ax1.set_ylabel('Euros (€)')
    ax1.set_ylim(-5, 15)
    ax1.set_xlim(pd.to_datetime("2025-05-01 00:00"), pd.to_datetime("2025-05-07 23:00"))
    ax1.set_xlabel('Fecha')
    ax1.tick_params(axis='x', rotation=45)

    # Eje Y derecho - Kilovatios
    ax2 = ax1.twinx()
    ax2.plot(df_temp['fecha'], df_temp['produccion solar'], label='Producción solar (kW)', color='tab:green')
    ax2.set_ylabel('Kilovatios (kW)')
    ax2.set_ylim(0, 25)

    # Leyenda
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    plt.title('Gastos Energéticos y Producción Solar (Simulación horaria del 1 al 7 mayo 2025)')
    plt.tight_layout()
    plt.savefig(f"graficos_semanales/frames_1_simulado/frame_{i:03d}.png")
    plt.close()

# Crear GIF
with imageio.get_writer("graficos_semanales/grafico_con_placas.gif", mode='I', duration=500) as writer:
    for i in range(1, len(df_subset) + 1):
        filename = f"graficos_semanales/frames_1_simulado/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)


C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\3120360873.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_subset['fecha'] = pd.date_range(start="2025-05-01 00:00", periods=168, freq='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\3120360873.py:63: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os

# Cargar datos
df = pd.read_csv("energy_results_sin_placas.csv")

# Renombrar columnas
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI'
})

# Seleccionar primeras 168 filas (simulación de 168 horas)
df_subset = df.iloc[:168].copy().reset_index(drop=True)

# Asignar fechas simuladas de 1 mayo 2025 00:00 a 7 mayo 2025 23:00
df_subset['fecha'] = pd.date_range(start="2025-05-01 00:00", periods=168, freq='H')

# Crear carpeta para frames
os.makedirs("graficos_semanales/frames_1_simulado_sin_placas", exist_ok=True)

# Generar frames animados
for i in range(1, len(df_subset) + 1):
    df_temp = df_subset.iloc[:i]

    fig, ax = plt.subplots(figsize=(12, 6))

    # Eje Y izquierdo - Euros
    ax.plot(df_temp['fecha'], df_temp['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
    ax.plot(df_temp['fecha'], df_temp['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
    ax.set_ylabel('Euros (€)')
    ax.set_ylim(-5, 15)
    ax.set_xlim(pd.to_datetime("2025-05-01 00:00"), pd.to_datetime("2025-05-07 23:00"))
    ax.set_xlabel('Fecha')
    ax.tick_params(axis='x', rotation=45)

    # Leyenda
    ax.legend(loc='upper left')
    plt.title('Gastos Energéticos SIN Placas (Simulación horaria del 1 al 7 mayo 2025)')
    plt.tight_layout()
    frame_path = f"graficos_semanales/frames_1_simulado_sin_placas/frame_{i:03d}.png"
    plt.savefig(frame_path)
    plt.close()

# Crear GIF
with imageio.get_writer("graficos_semanales/grafico_sin_placas.gif", mode='I', duration=1.0) as writer:
    for i in range(1, len(df_subset) + 1):
        filename = f"graficos_semanales/frames_1_simulado_sin_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)


C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\3054037937.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_subset['fecha'] = pd.date_range(start="2025-05-01 00:00", periods=168, freq='H')
C:\Users\anaig\AppData\Local\Temp\ipykernel_27508\3054037937.py:51: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
